In [1]:
# imports
import pandas as pd
from pandas import json_normalize
import sqlite3 as sq
import os
import requests
import json

In [2]:
#Key initialization
#set the keys
foursquare_id = os.environ["FOURSQUARE_API_KEY"]
foursquare_secret = os.environ["FOURSQUARE_SECRET"]

In [3]:
#set base url
url = "https://api.foursquare.com/v3/places/search?ll=43.656112128112106%2C-79.3802357750855&radius=1000&categories=10039&fields=rating%2Cname%2Cdistance%2Clocation"

#set header variable
headers = {"accept": "application/json",
           "authorization": foursquare_id}

In [4]:
#---Static-Variables-----

location = '43.656112128112106,-79.3802357750855'    #Young-Dundas Square
location2 = '43.725543525284145,-79.45215847331576'  #Yorkdale Mall
radius = 1000   #in meters

# #list of FS category ids to search
# cat_ids = [10039, 10041, 10037]

In [5]:
# #string variable for category portion of query
# catqry = "&categories="

#Turn Categories into string for query. %2C represents a comma in urls.
#for cat in range(0, len(cat_ids)):
#   catqry += "%2C" 
#     catqry += str(cat_ids[cat])

In [6]:
def get_venues(urlstr):
    response = requests.get(urlstr, headers=headers)
    response.raise_for_status()
    if response.status_code != 204:
        return response

In [7]:
#query building function to be called in main pull request loop
# def build_query(latlong):
#     #make lat and long variables from parameter
#     splitll = latlong.split(',')
#     lat = f"%3D{splitll[0]}"
#     long = f"%2C{splitll[1]}"
    
#     #build out query using parameters.
#     query = (f"ll={lat}{long}&radius={radius}{catqry}")
#     fullurl = str(url+query)
#     return fullurl
#     #get_venues(fullurl)

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [8]:
data = get_venues(url)

Parse through the response to get the POI details you want (rating, name, location, etc)

In [9]:
#turn data into a sorted by keys str
parsed_data = data.json()

In [10]:
type(parsed_data)

dict

Put your parsed results into a DataFrame

In [11]:
#Convert JSON to DataFrame using json_normalize()
df = pd.json_normalize(parsed_data['results'])

In [12]:
print('Music Venues within 1km of Young-Dundas Square (in meters):')
sort_fs_df = df[['name', 'location.formatted_address', 'rating']].sort_values('rating', ascending=False).rename(columns={'location.formatted_address': 'address'}).fillna(0.0)
sort_fs_df['rating'] = sort_fs_df['rating']*0.5
fs_df = sort_fs_df.reset_index(drop=True)
fs_df

Music Venues within 1km of Young-Dundas Square (in meters):


,name,address,rating
0,Elgin & Winter Garden Theatre Ctr,"189 Yonge St (btwn Queen St E & Shuter St), To...",4.50
1,The Four Seasons Centre for the Performing Arts,"145 Queen St W (at University Ave.), Toronto O...",4.45
2,Jazz Bistro,251 Victoria St (btwn Dundas St. E & Shuter St...,4.25
3,Canadian Opera Co,"145 Queen St W (University), Toronto ON M5H 4G1",4.00
4,Betty Oliphant Theatre,"400 Jarvis St, Toronto ON M4Y 2G6",3.95
5,Buddies in Bad Times Theatre,"12 Alexander St (Yonge St), Toronto ON M4Y 1B4",3.90
6,Bar Plus,"2Nd Floor, Toronto ON M5B 1S5",3.10
7,Bluma Appel Theatre,"27 Front St E, Toronto ON M5E 1B4",3.10
8,Ryerson Theatre,"43 Gerrard St E (btwn Yonge & Church), Toronto...",3.00
9,Fusion Rock Radio,"10 Dundas St E (at Yonge St.), Toronto ON M5B 2G9",0.00


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [13]:
#yelp api initialize
yelp_api = os.environ["YELP_API_KEY"]
yelp_headers = {"accept": "application/json",
           "Authorization": f"BEARER {yelp_api}"}

#yelp query Variables
yelp_url = "https://api.yelp.com/v3/businesses/search?latitude=43.656112128112106&longitude=-79.3802357750855&radius=1000&categories=musicvenues&="
#yelp data pull
yelp_response = requests.get(yelp_url, headers=yelp_headers)


Parse through your result and get POI details

In [14]:
yelp_data = yelp_response.json()

Put your parsed results into a DataFrame

In [44]:
df = pd.json_normalize(yelp_data['businesses'])
sort_yp_df = df[['name', 'location.display_address', 'rating']].sort_values('rating', ascending=False).rename(columns={'location.display_address': 'address'})
yp_df = sort_yp_df.reset_index(drop=True)


In [46]:
yp_df['address'] = yp_df['address'].apply(lambda x: str(x))

In [47]:
yp_df

,name,address,rating
0,Bar Cathedral,"['54 The Esplanade', 'Toronto, ON M5E 1A6', 'C...",5.0
1,Massey Hall,"['178 Victoria Street', 'Toronto, ON M5B 1T7',...",4.0
2,The Reservoir Lounge,"['52 Wellington Street E', 'Toronto, ON M5E 1...",4.0
3,Ryerson Theatre,"['43 Gerrard Street E', 'Toronto, ON M5G 2A7',...",4.0
4,The Cavern Bar,"['76 Church Street', 'Toronto, ON M5C 2G1', 'C...",4.0
5,Waterfall Stage,"['First Canadian Place', '100 King St W', 'Tor...",4.0
6,C'est What,"['67 Front Street E', 'Toronto, ON M5E 1B5', '...",3.5
7,Jazz Bistro,"['251 Victoria Street', 'Toronto, ON M5B 1T8',...",3.5
8,Adelaide Hall,"['250 Adelaide Street W', 'Toronto, ON M5H 1X6...",3.5
9,The Sultan's Tent & Cafe Moroc,"['49 Front Street E', 'Toronto, ON M5E 1B3', '...",3.0


In [17]:
# yp_df2 = yp_df.explode(['address']).reset_index(drop=True)

In [18]:
# strJoin = lambda x:",".join(x.astype(str))   
# yp_df3 = yp_df2.groupby(['name', 'rating']).agg({"address":strJoin})

In [19]:
# yp_df3

# Google (stretch)

Use the same process as the first two APIs

In [21]:
#Google Places API
google_api = os.environ["GOOGLE_API_KEY"]

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [48]:
# #create database
conn = sq.connect('../data/eda_data')
# c = conn.cursor()

# #initialize the table
# #c.execute('CREATE TABLE IF NOT EXISTS locations (name text, rating real, address text)')
# conn.commit()

In [49]:
# #send dataframes to sql
yp_df.to_sql('locations', conn, if_exists='replace', index=False)

12

In [50]:
fs_df.to_sql('locations', conn, if_exists='append', index=False)

10

In [52]:
result = pd.read_sql('SELECT rating, name\
                      FROM locations\
                      ORDER BY rating DESC\
                      LIMIT 10', conn)
print(result)

   rating                                             name
0    5.00                                    Bar Cathedral
1    4.50                Elgin & Winter Garden Theatre Ctr
2    4.45  The Four Seasons Centre for the Performing Arts
3    4.25                                      Jazz Bistro
4    4.00                                      Massey Hall
5    4.00                             The Reservoir Lounge
6    4.00                                  Ryerson Theatre
7    4.00                                   The Cavern Bar
8    4.00                                  Waterfall Stage
9    4.00                                Canadian Opera Co


Get the top 10 restaurants according to their rating

# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).